# モデルチューニング
特徴量選択・ラッパー法による精度向上に加え、LGBMのモデルを最適化することで精度向上を行う。

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import gc

# 分布確認に使う
# import pandas_profiling as pdp
# 可視化
import matplotlib.pyplot as plt
# 前処理、特徴量作成 - sklearnを使う
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
# モデリング・精度と評価指標
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
#LGBM
import lightgbm as lgb

#どうでもいい警告は無視
import warnings
warnings.filterwarnings("ignore")

# NOTE matplotでの日本語文字化けを解消
#pip install japanize-matplotlib
import japanize_matplotlib
%matplotlib inline

# データ確認

In [ ]:
df_train = pd.read_csv("titanic_datasets/train.csv")
df_train.head()

In [ ]:
print("データ形状：")
print(df_train.shape)

print("データ数：")
print(len(df_train))

print("データのコラム数")
print(len(df_train.columns))

In [ ]:
print("データ型一覧")
df_train.info()

In [ ]:
# 量的変数の要約統計
df_train.describe().T

In [ ]:
#質的変数の度数
df_train.describe(exclude="number").T

# データ前処理

In [ ]:
df_train.isnull().sum()

In [ ]:
#性別(sex)→0,1
# 古典的な男女の質的変数など、ダミー化した情報が1つで十分な場合は多重共線性対策で1つのみ情報を残す
df_train = pd.get_dummies(df_train, columns = ["Sex"], drop_first=True)

#出港地(Embarked)→0,1,2
#df_train['Embarked'].fillna(('S'), inplace=True)
df_train = pd.get_dummies(df_train, columns = ["Embarked"])

#運賃(Fare)→欠損値は平均値にする
df_train['Fare'].fillna(np.mean(df_train['Fare']), inplace=True)
#運賃(Fare)のヒストグラムを描写。ポアソン分布となる説明変数はビニングすると精度が向上する事が分かっている
#df_train['Fare'].hist(bins=50)

# NOTE 境界値を指定したビニング
bins_Fare = [0,50,100,200,300,1000]
# T_Bil列を分割し、0始まりの連番でラベル化した結果をX_cutに格納する
X_cut, bin_indice = pd.cut(df_train["Fare"], bins=bins_Fare, retbins=True, labels=False)
# bin分割した結果をダミー変数化 (prefix=X_Cut.nameは、列名の接頭語を指定している)
X_dummies = pd.get_dummies(X_cut, prefix=X_cut.name)
# 元の説明変数のデータフレーム(X)と、ダミー変数化した結果(X_dummies)を横連結
df_train = pd.concat([df_train, X_dummies], axis=1)

#年齢(Age)→欠損値は平均値・標準偏差を使って正規乱数で埋め合わせる
age_avg = df_train['Age'].mean()
age_std = df_train['Age'].std()
df_train['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)

# 特徴量：Familysizeを作成(Parch, Sibspを足した値)
df_train['FamilySize'] = df_train['Parch'] + df_train['SibSp'] + 1
df_train['FamilySize_bin'] = 'big'
df_train.loc[df_train['FamilySize']==1,'FamilySize_bin'] = 'alone'
df_train.loc[(df_train['FamilySize']>=2) & (df_train["FamilySize"]<=4),'FamilySize_bin'] = 'small'
df_train.loc[(df_train['FamilySize']>=5) & (df_train["FamilySize"]<=7),'FamilySize_bin'] = 'mediam'

# 特徴量：TicketFreq - Ticket頻度をチケットを表す量的変数とする
df_train.loc[:, 'TicketFreq'] = df_train.groupby(['Ticket'])['PassengerId'].transform('count')

# 特徴量：HonorificをNameから抽出
df_train['honorific'] = df_train['Name'].map(lambda x: x.split(', ')[1].split('. ')[0])
df_train['honorific'].replace(['Col','Dr', 'Rev'], 'Rare',inplace=True) #少数派の敬称を統合
df_train['honorific'].replace('Mlle', 'Miss',inplace=True) #Missに統合
df_train['honorific'].replace('Ms', 'Miss',inplace=True) #Missに統合

# 特徴量：Cabin_ini - Cabinの頭文字を抽出
df_train['Cabin_ini'] = df_train['Cabin'].map(lambda x:str(x)[0])
df_train['Cabin_ini'].replace(['G','T'], 'Rare',inplace=True)

# FamilySize_bin, honorific, Cabin_iniをすべてダミー化する
df_train = pd.get_dummies(df_train, columns = ["FamilySize_bin", "honorific", "Cabin_ini"])

print("コラム名:")
print(df_train.columns)

print("コラム数:")
print(len(df_train.columns))

# 特徴量選択

In [ ]:
target_col = 'Survived'
drop_col = ['PassengerId','Survived', 'Name', 'Fare', 'Ticket', 'Cabin', 'Parch', 'FamilySize', 'SibSp']

y = df_train[target_col]
X = df_train.drop(drop_col , axis=1)

In [ ]:
## ラッパー法(GBDT)で説明変数を削減

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor

# estimatorとしてGBDTを使用。特徴量を20個選択
selector = RFE(GradientBoostingRegressor(n_estimators=100, random_state=10), n_features_to_select=20)
selector.fit(X, y)
mask = selector.get_support()
#print(X.feature_names)
print(mask)

# 選択した特徴量の列のみ取得
X_selected = selector.transform(X)
print("X.shape={}, X_selected.shape={}".format(X.shape, X_selected.shape))

In [ ]:
list = []
not_selected = []
columns = X.columns

for i in range(0, len(mask)):
    
    value = mask[i]
    
    if (value == True):
        list.append(columns[i])
    else:
        not_selected.append(columns[i])

print("選択された20の特徴量：")
print(list)

print("選択されなかった特徴量：")
print(not_selected)

# モデル作成・学習
今回は学習段階でのハイパーパラメータを最適化し,汎用精度の向上を目指します。
モデルの最適化には、大きく2つの手法がある
- 手動チューニング - カンを頼りにパラメータを変える
- 自動チューニング - Optuna, scikit-optimizeなどを利用して自動で分類精度の高いパラメータを探索する

In [ ]:
X_train, y_train, id_train = X[list], df_train[["Survived"]], df_train[["PassengerId"]]

# データ形状が問題ないか判定
print(X_train.shape)
print(y_train.shape)
print(id_train.shape)

In [ ]:
X_train.isnull().sum()

In [ ]:
X_train.head(10)

→true・falseを0・1に変換、scikit-learnの別のアルゴリズムを活用してアンサンブル学習をできるようにする。

In [ ]:
# ホールドアウト検証 - 学習用・テスト用に分割
X_tr, X_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, stratify=y_train, random_state=123)

print("学習用・訓練用データの形状：")
print(X_tr.shape)
print(y_tr.shape)
print(X_va.shape)
print(y_va.shape)
    
# データのラベルに偏りがないか表示
y_train_mean = y_train["Survived"].mean()
y_tr_mean = y_tr["Survived"].mean()
y_va_mean = y_va["Survived"].mean()
    
print("全体の生存者割合：")
print(y_train_mean)
print("学習用データの生存者割合：")
print(y_tr_mean)
print("検証用データの生存者割合：")
print(y_va_mean)

In [ ]:
# LGBMのパラメータ(学習しないパラメータのみこの段階で定義する)
params_base = {"boosting_type":"gbdt",
          "objective":"binary",
          "metric":"auc",
          "learning_rate":0.02,
          "num_leaves":16,
          "n_estimators":100000,
          "bagging_freq:":1,
          "seed": 123,
          }

In [ ]:
pip install optuna

In [ ]:
import optuna

# 最適パラメータの探索
optunaでベイズ最適化によるチューニングを行う。  scikit-learnならGridSearchCVでグリッドサーチをするのもあり。
基本探索するべきパラメータが5以上ある場合はベイズ最適によるチューニングのほうが効率的。

In [ ]:
# チューニングで探索する最適パラメータ
def objective(trial):
    params_tuning = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 256),
        "min_data_in_leaf":trial.suggest_int("min_data_in_leaf", 5,200),
        "min_sum_hessian_in_leaf":trial.suggest_float("min_sum_hessian_in_leaf", 0.00001, 0.01, log = True),
        "feature_fraction":trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction":trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "lambda_l1":trial.suggest_float("lambda_l1", 0.01, 10.0, log = True),
        "lambda_l2":trial.suggest_float("lambda_l2", 0.01, 10.0, log = True)
    }
    
    #tuningにbaseの値を加える
    params_tuning.update(params_base)
    
    #モデル学習(ベイズ最適化)
    list_metrics=[]
    
    # ホールドアウト検証 - 学習用・テスト用の分割を1通り決める
    X_tr, X_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, stratify=y_train, random_state=123)

        
    model = lgb.LGBMClassifier(**params_tuning)
        
    model.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr),(X_va, y_va)], 
                    #early_stopping_rounds=100, 
                    #verbose=0
                    )
    y_va_pred = model.predict_proba(X_va)[:, 1]
    metric_va = accuracy_score(y_va, np.where(y_va_pred >= 0.5, 1, 0))
    
    return metric_va

In [ ]:
# 探索を実行
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction="maximize")
study.optimize(objective, n_trials=10)

In [ ]:
# 探索で得られた結果を確認
trial = study.best_trial
print("最も高い精度：")
print(trial.value)

print("最も高い精度となるパラメータ：")
print(trial.params)

In [ ]:
#param_best = trial.params
param_best = {'num_leaves': 219, 'min_data_in_leaf': 146, 'min_sum_hessian_in_leaf': 0.0006808799287054756, 'feature_fraction': 0.8612216912851107, 'bagging_fraction': 0.6614794569265892, 'lambda_l1': 0.1217211275847546, 'lambda_l2': 0.048393796358465295}

In [ ]:
# 元のパラメータに最高精度が得られたパラメータを足す
#param_best = trial.params
params_best = {'num_leaves': 219, 'min_data_in_leaf': 146, 'min_sum_hessian_in_leaf': 0.0006808799287054756, 'feature_fraction': 0.8612216912851107, 'bagging_fraction': 0.6614794569265892, 'lambda_l1': 0.1217211275847546, 'lambda_l2': 0.048393796358465295}
params_best.update(params_base)
display(params_best)

In [ ]:
# LGBMのモデルを学習開始
model = lgb.LGBMClassifier(**params_best)
model.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr),(X_va, y_va)])

In [ ]:
# AUC値に加え精度を算出する
y_tr_pred = model.predict(X_tr)
y_va_pred = model.predict(X_va)

metric_tr = accuracy_score(y_tr, y_tr_pred)
metric_va = accuracy_score(y_va, y_va_pred)
print("モデル精度:")
print("学習精度")
print(metric_tr)
print("検証精度")
print(metric_va)

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

reg = LazyClassifier(ignore_warnings=True, random_state=1121, verbose=False,predictions=True)
models, predictions = reg.fit(X_tr, X_va, y_tr, y_va) 

print("モデルの精度・評価指標：")
display(models)
print("テストデータの予測値：")
display(predictions)

In [ ]:
y_pred=model.predict(X_va)
y_pred[:5]

# 精度検証 - 混合行列

In [ ]:
# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1"])
disp.plot()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

"""
適合率は予測されたラベルのうち、どれだけが正解だったか
再現率は正解のラベルのうち、どれだけそのラベルを予測できたか

一般的に誤検出を少なくしたい場合には適合率。取りこぼしを少なくしたい場合には再現率を重視します。

医療等に機械学習を活用する等「取りこぼしが多いと命に関わるような重大な問題に繋がる」といったような場合には、再現率を評価指標に使用するのが適切です。
"""

#適合率 - TP+TN / (TP+FP+TN+FN) - 単純な正答率
print("正答率")
print(accuracy_score(y_pred, y_va))

#適合率 - TP / (TP+FP) - 予測されたラベルに対する実際に正解となるサンプル
print("適合率")
print(precision_score(y_pred, y_va))

#再現率 - TP / (TP+FN) - 本来正しく予測した場合に対し実際に正解となるサンプル
print("再現率")
print(recall_score(y_pred, y_va))

print("F値")
print(f1_score(y_pred, y_va))

# LazyPredict

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

reg = LazyClassifier(ignore_warnings=True, random_state=1121, verbose=False,predictions=True)
models, predictions = reg.fit(X_tr, X_va, y_tr, y_va) 

print("モデルの精度・評価指標：")
display(models)
print("テストデータの予測値：")
display(predictions)

# 提出用データを作成

In [ ]:
df_test = pd.read_csv("titanic_datasets/test.csv")
df_test.head(5)

In [ ]:
#性別(sex)→0,1
# 古典的な男女の質的変数など、ダミー化した情報が1つで十分な場合は多重共線性対策で1つのみ情報を残す
df_test = pd.get_dummies(df_test, columns = ["Sex"], drop_first=True)

#出港地(Embarked)→0,1,2、不明の値は仮にS=0にする
df_test['Embarked'].fillna(('S'), inplace=True)
df_test = pd.get_dummies(df_test, columns = ["Embarked"])

#運賃(Fare)→欠損値は平均値にする
df_test['Fare'].fillna(np.mean(df_test['Fare']), inplace=True)
#運賃(Fare)のヒストグラムを描写。ポアソン分布となる説明変数はビニングすると精度が向上する事が分かっている
df_test['Fare'].hist(bins=50)

# NOTE 境界値を指定したビニング
bins_Fare = [0,50,100,200,300,1000]
# T_Bil列を分割し、0始まりの連番でラベル化した結果をX_cutに格納する
X_cut, bin_indice = pd.cut(df_test["Fare"], bins=bins_Fare, retbins=True, labels=False)
# bin分割した結果をダミー変数化 (prefix=X_Cut.nameは、列名の接頭語を指定している)
X_dummies = pd.get_dummies(X_cut, prefix=X_cut.name)
# 元の説明変数のデータフレーム(X)と、ダミー変数化した結果(X_dummies)を横連結
df_test = pd.concat([df_test, X_dummies], axis=1)

#年齢(Age)→欠損値は平均値・標準偏差を使って正規乱数で埋め合わせる
age_avg = df_test['Age'].mean()
age_std = df_test['Age'].std()
df_test['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)

# 特徴量：Familysizeを作成(Parch, Sibspを足した値)
df_test['FamilySize'] = df_test['Parch'] + df_test['SibSp'] + 1 #ALLデータ
# FamilySizeを離散化
df_test['FamilySize_bin'] = 'big'
df_test.loc[df_test['FamilySize']==1,'FamilySize_bin'] = 'alone'
df_test.loc[(df_test['FamilySize']>=2) & (df_test["FamilySize"]<=4),'FamilySize_bin'] = 'small'
df_test.loc[(df_test['FamilySize']>=5) & (df_test["FamilySize"]<=7),'FamilySize_bin'] = 'mediam'

# 特徴量：TicketFreq - Ticket頻度をチケットを表す量的変数とする
df_test.loc[:, 'TicketFreq'] = df_test.groupby(['Ticket'])['PassengerId'].transform('count')

# 特徴量：HonorificをNameから抽出
df_test['honorific'] = df_test['Name'].map(lambda x: x.split(', ')[1].split('. ')[0])
df_test['honorific'].replace(['Col','Dr', 'Rev'], 'Rare',inplace=True) #少数派の敬称を統合
df_test['honorific'].replace('Mlle', 'Miss',inplace=True) #Missに統合
df_test['honorific'].replace('Ms', 'Miss',inplace=True) #Missに統合

# 特徴量：Cabin_ini - Cabinの頭文字を抽出
df_test['Cabin_ini'] = df_test['Cabin'].map(lambda x:str(x)[0])
df_test['Cabin_ini'].replace(['G','T'], 'Rare',inplace=True) #少数派のCabin_iniを統合

df_test = pd.get_dummies(df_test, columns = ["FamilySize_bin", "honorific", "Cabin_ini"])

In [ ]:
test_feature = df_test[list]
test_feature.columns

In [ ]:
y_pred = model.predict(test_feature)

# NOTE 予測結果を1次元ベクトルにする
y_pred = np.squeeze(y_pred)
y_pred.shape

In [ ]:
y_pred[(y_pred >= 0.5)] = 1
y_pred[(y_pred < 0.5)] = 0
y_pred = np.array(y_pred, dtype='int')

y_pred

In [ ]:
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': y_pred})
output.to_csv('submission_LGBM_bayes_optimized_.csv', index=False)